In [15]:
%reload_ext autoreload
%autoreload 2
from IPython.display import display

In [16]:
import jsonlines
import glob
from collections import defaultdict

In [17]:
for f in glob.glob("../samples/anomaly_patterns/*.jsonl"):
    with jsonlines.open(f) as reader:
        aggr = defaultdict(int)
        aggr2 = defaultdict(int)
        for obj in reader:
            aggr[obj["anomaly_pattern"]] += 1
            aggr2[obj["anomaly_position"]] += 1
    if len(aggr) < 1:
        continue
    display(f, aggr, aggr2)

'../samples/anomaly_patterns/anomaly_patterns_20221018-105001.jsonl'

defaultdict(int,
            {'Single spike': 62,
             'Other normal': 172,
             'White noise': 34,
             'Level shift up': 71,
             'Level shift down': 180,
             'Transient level shift down': 20,
             'Sudden increase': 16,
             'Multiple spikes': 23,
             'Transient level shift up': 19,
             'Multiple dips': 10,
             'Steady increase': 21,
             'Single dip': 69,
             'Steady decrease': 2,
             'Fluctuations': 11,
             'Sudden decrease': 6})

defaultdict(int,
            {'anomaly_during_fault': 465,
             'no_anomaly': 206,
             'anomaly_outside_fault': 45})

'../samples/anomaly_patterns/anomaly_patterns_20221017-150244.jsonl'

defaultdict(int,
            {'Level shift down': 2,
             'White noise': 1,
             'Other normal': 9,
             'Single spike': 3})

...

defaultdict(int, {'Level shift down': 1})

defaultdict(int, {'anomaly_during_fault': 1})

'../samples/anomaly_patterns/anomaly_patterns_20221017-150108.jsonl'

defaultdict(int, {'Level shift down': 1})

defaultdict(int, {'anomaly_during_fault': 1})

In [18]:
import sys
sys.path.append('../')

from tsdr import unireducer

In [35]:
fpath = "../samples/anomaly_patterns/anomaly_patterns_20221018-105001.jsonl"

samples = defaultdict(list)
with jsonlines.open(fpath) as reader:
    for obj in reader:
        apos, apattern = obj["anomaly_position"], obj["anomaly_pattern"]
        if apos == "no_anomaly" or apattern in ["White noise", "Other normal"]:
            samples["normal"].append(obj)
        else:
            samples[f"{apos}/{apattern}"].append(obj)

In [20]:
import numpy as np
import pandas as pd
import random
import scipy.interpolate

In [29]:
NUM_SAMPLES_BY_PATTERN = 10
FAILURE_DETECT_IDX = 99
NUM_DATAPOINTS = 120
NUM_FIRST_NAN = 4

In [30]:
def eval_time_series(pattern_and_pos, obj):
    _ts = np.array(obj["time_series"])
    ts = _ts[~np.isnan(_ts)]
    ts = scipy.interpolate.interp1d(
        x=np.arange(NUM_DATAPOINTS - len(ts), NUM_DATAPOINTS), y=ts, kind="cubic", fill_value="extrapolate",
    )(np.arange(0, NUM_DATAPOINTS))
    rim_res = unireducer.residual_integral_model(ts, **{
        "step1_residual_integral_threshold": 20,
        "step1_residual_integral_change_start_point": False,
    })
    # In [Wu+, ISSRE21] PatternMatcher, $m[t-l_1:t]$（test window SA）と$m[t-l_1-l_2:t-l_1]$（normal window SN） l1=10, l2=30
        # t is failure detection time.
    ks_res = unireducer.two_samp_test_model(ts, **{
        "step1_two_samp_test_alpha": 0.05,
        "step1_two_samp_test_seg_idx": FAILURE_DETECT_IDX - 10*4,
        "step1_two_samp_test_method": "ks",
    })
    ediag_res = unireducer.two_samp_test_model(ts, **{
        "step1_two_samp_test_alpha": 0.05,
        "step1_two_samp_test_seg_idx": FAILURE_DETECT_IDX,
        "step1_two_samp_test_method": "e-diagnosis",
    })
    fi_res = unireducer.fluxinfer_model(ts, **{
        "step1_fluxinfer_sigma_threshold": 3, 
    })
    eval_results = []
    for method, res in zip(["resid_sum", "ks_test", "e_diagnosis", "fluxinfer"], [rim_res, ks_res, ediag_res, fi_res]):
        eval_results.append({
            "method": method,
            "detection": "anomaly" if res.has_kept else "normal",
            "ground_truth": "anomaly" if pattern_and_pos != "normal" else "normal",
            "confusion": "TP" if res.has_kept and pattern_and_pos != "normal" else "TN" if not res.has_kept and pattern_and_pos == "normal" else "FP" if res.has_kept and pattern_and_pos == "normal" else "FN",
            "raw_pattern": obj["anomaly_pattern"],
            "raw_position": obj["anomaly_position"],
        })
    return eval_results

In [54]:
eval_results: list = []
# The number of "normal" samples should be equal to the number of "anomaly" samples.

# anomaly pattern
num_anomaly_samples: int = 0
for pattern, objs in [(k, v) for k, v in samples.items() if k != "normal"]:
    random.shuffle(objs)
    head = NUM_SAMPLES_BY_PATTERN if len(objs) > NUM_SAMPLES_BY_PATTERN else len(objs)
    num_anomaly_samples += head
    for obj in objs[:head]:
        eval_results += eval_time_series(pattern, obj)

# normal pattern
random.shuffle(samples["normal"])
for obj in samples["normal"][:num_anomaly_samples]:
    eval_results += eval_time_series("normal", obj)

df = pd.DataFrame(eval_results)
pd.set_option('display.max_rows', 10)
df

TypeError: list indices must be integers or slices, not str

In [51]:
df.groupby(["method", "ground_truth"]).size()

method       ground_truth
e_diagnosis  anomaly         146
             normal          209
fluxinfer    anomaly         146
             normal          209
ks_test      anomaly         146
             normal          209
resid_sum    anomaly         146
             normal          209
dtype: int64

In [22]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

In [44]:
eval_df = df.groupby("method").apply(lambda x: x.groupby("confusion").size())
eval_df

confusion,FN,FP,TN,TP
method,,,,
e_diagnosis,35,7,13,111
fluxinfer,60,15,5,86
ks_test,48,11,9,98
resid_sum,5,12,8,141


In [24]:
# https://en.wikipedia.org/wiki/Precision_and_recall
def calc_scores(X: pd.Series) -> pd.Series:
    X["TP"] = X["TP"] if "TP" in X.index else 0
    X["TN"] = X["TN"] if "TN" in X.index else 0
    X["FN"] = X["FN"] if "FN" in X.index else 0
    X["FP"] = X["FP"] if "FP" in X.index else 0
    return pd.Series({
        "accuracy": (X["TP"] + X["TN"]) / (X["TP"] + X["TN"] + X["FP"] + X["FN"]),
        "precision": X["TP"] / (X["TP"] + X["FP"]),
        "recall": X["TP"] / (X["TP"] + X["FN"]),
        "f1-score": 2 * X["TP"] / (2 * X["TP"] + X["FP"] + X["FN"]),
    })

In [45]:
eval_df.T.apply(calc_scores)

method,e_diagnosis,fluxinfer,ks_test,resid_sum
accuracy,0.747,0.548,0.645,0.898
precision,0.941,0.851,0.899,0.922
recall,0.760,0.589,0.671,0.966
f1-score,0.841,0.696,0.769,0.943


In [26]:
eval_df_by_label = df.groupby(by=["method", "raw_pattern", "raw_position"]).apply(lambda x: x.groupby("confusion").size())
display(eval_df_by_label.to_frame())

0
method      raw_pattern                raw_position          confusion    
e_diagnosis Fluctuations               anomaly_during_fault  TP          7
                                       anomaly_outside_fault TP          3
            Level shift down           anomaly_during_fault  TP         10
                                       anomaly_outside_fault FN          1
                                                             TP          1
            Level shift up             anomaly_during_fault  TP         10
            Multiple dips              anomaly_during_fault  FN          1
                                                             TP          9
            Multiple spikes            anomaly_during_fault  FN          3
                                                             TP          7
                                       anomaly_outside_fault FN          7
            Other normal               no_anomaly            FP          5
                                                             TN          5
            Single dip                 anomaly_during_fault  FN          3
                                                             TP          7
                                       anomaly_outside_fault FN          3
                                                             TP          2
            Single spike               anomaly_during_fault  FN          3
                                                             TP          7
                                       anomaly_outside_fault FN          8
                                                             TP          2
            Steady decrease            anomaly_during_fault  TP          2
            Steady increase            anomaly_during_fault  TP         10
            Sudden decrease            anomaly_during_fault  TP          6
            Sudden increase            anomaly_during_fault  FN          1
                                                             TP          9
            Transient level shift down anomaly_during_fault  TP         10
            Transient level shift up   anomaly_during_fault  TP         10
                                       anomaly_outside_fault FN          4
fluxinfer   Fluctuations               anomaly_during_fault  FN          3
                                                             TP          4
                                       anomaly_outside_fault FN          1
                                                             TP          2
            Level shift down           anomaly_during_fault  TP         10
                                       anomaly_outside_fault TP          2
            Level shift up             anomaly_during_fault  TP         10
            Multiple dips              anomaly_during_fault  FN          3
                                                             TP          7
            Multiple spikes            anomaly_during_fault  FN          6
                                                             TP          4
                                       anomaly_outside_fault FN          6
                                                             TP          1
            Other normal               no_anomaly            FP          8
                                                             TN          2
            Single dip                 anomaly_during_fault  FN          5
                                                             TP          5
                                       anomaly_outside_fault FN          4
                                                             TP          1
            Single spike               anomaly_during_fault  FN          2
                                                             TP          8
                                       anomaly_outside_fault FN          8
                                                             TP          2
            Steady decr

In [27]:
np.seterr(divide='ignore', invalid='ignore')
eval_df_score = df.groupby(by=["method", "raw_pattern", "raw_position"]).apply(
    lambda x: calc_scores(x.groupby("confusion")["confusion"].size()),
)
display(eval_df_score)

accuracy  precision  recall  f1-score
method      raw_pattern                raw_position                                                
e_diagnosis Fluctuations               anomaly_during_fault     1.000      1.0      1.000    1.000 
                                       anomaly_outside_fault    1.000      1.0      1.000    1.000 
            Level shift down           anomaly_during_fault     1.000      1.0      1.000    1.000 
                                       anomaly_outside_fault    0.500      1.0      0.500    0.667 
            Level shift up             anomaly_during_fault     1.000      1.0      1.000    1.000 
            Multiple dips              anomaly_during_fault     0.900      1.0      0.900    0.947 
            Multiple spikes            anomaly_during_fault     0.700      1.0      0.700    0.824 
                                       anomaly_outside_fault    0.000      NaN      0.000    0.000 
            Other normal               no_anomaly               0.500      0.0        NaN    0.000 
            Single dip                 anomaly_during_fault     0.700      1.0      0.700    0.824 
                                       anomaly_outside_fault    0.400      1.0      0.400    0.571 
            Single spike               anomaly_during_fault     0.700      1.0      0.700    0.824 
                                       anomaly_outside_fault    0.200      1.0      0.200    0.333 
            Steady decrease            anomaly_during_fault     1.000      1.0      1.000    1.000 
            Steady increase            anomaly_during_fault     1.000      1.0      1.000    1.000 
            Sudden decrease            anomaly_during_fault     1.000      1.0      1.000    1.000 
            Sudden increase            anomaly_during_fault     0.900      1.0      0.900    0.947 
            Transient level shift down anomaly_during_fault     1.000      1.0      1.000    1.000 
            Transient level shift up   anomaly_during_fault     1.000      1.0      1.000    1.000 
                                       anomaly_outside_fault    0.000      NaN      0.000    0.000 
fluxinfer   Fluctuations               anomaly_during_fault     0.571      1.0      0.571    0.727 
                                       anomaly_outside_fault    0.667      1.0      0.667    0.800 
            Level shift down           anomaly_during_fault     1.000      1.0      1.000    1.000 
                                       anomaly_outside_fault    1.000      1.0      1.000    1.000 
            Level shift up             anomaly_during_fault     1.000      1.0      1.000    1.000 
            Multiple dips              anomaly_during_fault     0.700      1.0      0.700    0.824 
            Multiple spikes            anomaly_during_fault     0.400      1.0      0.400    0.571 
                                       anomaly_outside_fault    0.143      1.0      0.143    0.250 
            Other normal               no_anomaly               0.200      0.0        NaN    0.000 
            Single dip                 anomaly_during_fault     0.500      1.0      0.500    0.667 
                                       anomaly_outside_fault    0.200      1.0      0.200    0.333 
            Single spike               anomaly_during_fault     0.800      1.0      0.800    0.889 
                                       anomaly_outside_fault    0.200      1.0      0.200    0.333 
            Steady decrease            anomaly_during_fault     0.500      1.0      0.500    0.667 
            Steady increase            anomaly_during_fault     0.900      1.0      0.900    0.947 
            Sudden decrease            anomaly_during_fault     0.333      1.0      0.333    0.500 
            Sudden increase            anomaly_during_fault     0.700      1.0      0.700    0.824 
            Transient level shift down anomaly_during_fault     0.600      1.0      0.600    0.750 
            Transient level shift up   anomaly_during_fault   

## Only normal patterns

In [39]:
eval_results: list = []
for obj in samples["normal"]:
    eval_results.extend(eval_time_series("normal", obj))

normal_df = pd.DataFrame(eval_results)
pd.set_option('display.max_rows', 10)
normal_df

,method,detection,ground_truth,confusion,raw_pattern,raw_position
0,resid_sum,anomaly,normal,FP,White noise,no_anomaly
1,ks_test,anomaly,normal,FP,White noise,no_anomaly
2,e_diagnosis,anomaly,normal,FP,White noise,no_anomaly
3,fluxinfer,normal,normal,TN,White noise,no_anomaly
4,resid_sum,normal,normal,TN,Other normal,no_anomaly
...,...,...,...,...,...,...
831,fluxinfer,anomaly,normal,FP,Other normal,no_anomaly
832,resid_sum,anomaly,normal,FP,Other normal,no_anomaly
833,ks_test,normal,normal,TN,Other normal,no_anomaly
834,e_diagnosis,anomaly,normal,FP,Other normal,no_anomaly


In [40]:
eval_normal_df = normal_df.groupby("method").apply(lambda x: x.groupby("confusion").size())
eval_normal_df

confusion,FP,TN
method,,
e_diagnosis,107,102
fluxinfer,159,50
ks_test,114,95
resid_sum,137,72


In [41]:
eval_normal_df.apply(calc_scores, axis=1)

,accuracy,precision,recall,f1-score
method,,,,
e_diagnosis,0.488,0.0,NaN,0.0
fluxinfer,0.239,0.0,NaN,0.0
ks_test,0.455,0.0,NaN,0.0
resid_sum,0.344,0.0,NaN,0.0
